In [4]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-oh29z976/unsloth_40b0f7d7c25c4042b80525aca9a2f861
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-oh29z976/unsloth_40b0f7d7c25c4042b80525aca9a2f861
  Resolved https://github.com/unslothai/unsloth.git to commit 02ba33964d87e4ef003da6483d2f37528468f439
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 22.7 MB/s eta 0:

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importação das bibliotecas

In [5]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import pandas as pd
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/trn.json"
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Colab Notebooks/formatted_trn.json"
max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Preparação do Dataset

In [ ]:
import pandas as pd

# O dataset possuía algumas linhas "corrompidas" devido a aspas desalinhadas, por isso incluímos uma validação para ignorá-las
with open(r"/content/drive/MyDrive/Colab Notebooks/trn.json", "r", encoding="utf-8") as f:
    records = []
    for line in f:
        try:
            records.append(json.loads(line))
        except json.JSONDecodeError:
            continue  # ignora linha corrompida

df = pd.DataFrame(records)[["title","content"]]

print("Tamanho original:")
print(df.shape)

print("Tamanho após dropna:")
df =df.dropna()

print(df.shape)

# Limpeza:
print("Após remover content vazio:")
df = df[df["content"].str.strip() != ""]                # remove vazio/só espaços
print(df.shape)
print("Após remover content curto (< 20 caracteres):")
df = df[df["content"].str.len() >= 20]                  # remove < 20 caracteres
print(df.shape)

# Opcional: também limpar 'title'
df = df[df["title"].str.strip() != ""]

print("Linhas restantes:", len(df))

# Salvar dataset limpo
df.to_json("/content/drive/MyDrive/Colab Notebooks/trn_clean.json",
           orient="records", lines=True, force_ascii=False)

print(df.head())

Tamanho original:
(2248619, 2)
Tamanho após dropna:
(2248619, 2)
Após remover content vazio:
(1498718, 2)
Após remover content curto (< 20 caracteres):
(1480741, 2)
Linhas restantes: 1374157


Chamada do Foundation Model

Execução do Fine Tuning

Geração de Respostas

Conclusões e Considerações Finais